In [3]:
classes = [
    "(Vertigo) Paroxysmal Positional Vertigo", 
    "AIDS", 
    "Acne", 
    "Alcoholic hepatitis", 
    "Allergy", 
    "Arthritis", 
    "Bronchial Asthma", 
    "Cervical spondylosis", 
    "Chicken pox", 
    "Chronic cholestasis", 
    "Common Cold", 
    "Dengue", 
    "Diabetes", 
    "Dimorphic hemorrhoids (piles)",
    "Drug Reaction",
    "Fungal infection",
    "GERD",
    "Gastroenteritis",
    "Heart attack",
    "Hepatitis B",
    "Hepatitis C",
    "Hepatitis D",
    "Hepatitis E",
    "Hypertension",
    "Hyperthyroidism",
    "Hypoglycemia",
    "Hypothyroidism",
    "Impetigo",
    "Jaundice",
    "Malaria",
    "Migraine",
    "Osteoarthritis",
    "Paralysis (brain hemorrhage)",
    "Peptic ulcer disease",
    "Pneumonia",
    "Psoriasis",
    "Tuberculosis",
    "Typhoid",
    "Urinary tract infection",
    "Varicose veins",
    "Hepatitis A"
]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

/Users/pedrolourenco/Documents/Web3/Nillion-Med-Translator/nillion-med-translator/model/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Diagnosis Classification 

In [3]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("shanover/symps_disease_bert_v3_c41", cache_dir="./llm-tokeniser")
model = AutoModelForSequenceClassification.from_pretrained("shanover/symps_disease_bert_v3_c41", cache_dir="./llm-model")

# Input text
text = "I am feeling vomiting, breathlessness, and sweating"

# Tokenize and prepare input for the model
inputs = tokenizer(text, return_tensors="pt")

# Run inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Apply softmax to get probabilities
probs = F.softmax(logits, dim=1)

# Get the predicted class
predicted_class = torch.argmax(probs, dim=1).item()

print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probs}")

/Users/pedrolourenco/Documents/Web3/Nillion-Med-Translator/nillion-med-translator/model/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Predicted class: 18
Class probabilities: tensor([[7.4849e-03, 9.9875e-04, 4.6661e-03, 3.0691e-03, 6.3025e-03, 2.6628e-03,
         2.0095e-03, 4.3723e-03, 4.6710e-03, 1.7938e-03, 5.2376e-03, 3.1679e-03,
         2.3369e-03, 7.2384e-04, 3.6365e-03, 5.2172e-03, 1.6723e-02, 9.7272e-03,
         7.5583e-01, 1.5773e-03, 1.5026e-03, 9.3692e-03, 7.2156e-03, 5.1868e-02,
         3.1464e-03, 1.5201e-02, 2.9604e-03, 3.6804e-03, 5.1570e-03, 7.9963e-03,
         2.1252e-03, 2.3563e-03, 2.4780e-03, 2.2738e-03, 7.7190e-03, 4.0139e-03,
         2.0764e-03, 1.3460e-02, 3.5330e-03, 4.7164e-03, 4.9413e-03]])


In [12]:
import onnx

def convert_pytorch_to_onnx_with_tokenizer(model, tokenizer, max_length=128, onnx_file_path=None):
    """
    Converts a PyTorch model to ONNX format, using tokenizer output as input.

    Args:
    model (torch.nn.Module): The PyTorch model to be converted.
    tokenizer: The tokenizer used to preprocess the input.
    onnx_file_path (str): The file path where the ONNX model will be saved.
    max_length (int): Maximum sequence length for the tokenizer.

    Returns:
    None
    """
    model.eval()

    # Prepare dummy input using the tokenizer
    dummy_input = "This is a sample input text for ONNX conversion."
    inputs = tokenizer(
        dummy_input,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

    # Get the input names
    input_names = list(inputs.keys())
    input_names = ["input_ids", "attention_mask"]
    print(f"Input names: {input_names}")

    # # Create dummy inputs for ONNX export
    # dummy_inputs = tuple(encoded_input[name] for name in input_names)
    dynamic_axes = {name: {0: "batch_size"} for name in input_names}
    dynamic_axes.update({f"logits": {0: "batch_size"}})
    print(f"dynamic_axes: {dynamic_axes}")
    
    # Export the model
    torch.onnx.export(
        model,  # model being run
        tuple(inputs[k] for k in input_names),  # model inputs
        onnx_file_path,  # where to save the model
        export_params=True,  # store the trained parameter weights inside the model file
        opset_version=20,  # the ONNX version to export the model to
        do_constant_folding=True,  # whether to execute constant folding for optimization
        input_names=input_names,  # the model's input names
        output_names=["logits"],  # the model's output names
        dynamic_axes=dynamic_axes,
    )  # variable length axes

    print(f"Model exported to {onnx_file_path}")

    # Verify the exported model
    onnx_model = onnx.load(onnx_file_path)
    onnx.checker.check_model(onnx_model)
    print("ONNX model is valid.")
    return onnx_file_path, input_names


# convert_pytorch_to_onnx_with_tokenizer(model, tokenizer)

In [22]:
model_name = "medicalai/ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./classifier-tokenizer")
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(classes), 
    cache_dir="./classifier-model"
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
onnx_file_path, input_names = convert_pytorch_to_onnx_with_tokenizer(
        model, tokenizer, max_length=128, onnx_file_path="./saved-models/diagnosis-classifier.onnx"
)

Input names: ['input_ids', 'attention_mask']
dynamic_axes: {'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'logits': {0: 'batch_size'}}
Model exported to ./saved-models/diagnosis-classifier.onnx
ONNX model is valid.


In [13]:
import aivm_client as aic
aic.get_supported_models()

['BertTinySMS', 'LeNet5MNIST']

In [18]:
import aivm_client as aic
aic.upload_bert_tiny_model("./saved-models/diagnosis-classifier.onnx", "diagnosis-classifier")

In [19]:
#running inference secure inference
input_text = "I am feeling vomiting, breathlessness, and sweating"
tokenized_input = aic.tokenize(input_text,)
encrypted_input = aic.BertTinyCryptensor(tokenized_input[0], tokenized_input[1])
result = aic.get_prediction(encrypted_input, "diagnosis-classifier")
result

KeyboardInterrupt: 

In [ ]:
#upload model
MODEL_NAME = "diagnosis-classifier" # Name of the model to be used
aic.upload_bert_tiny_model("./saved_models/diagnosis_classifier.onnx", MODEL_NAME) 

In [23]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn.functional import softmax

# Load the BERT Tiny model and tokenizer
model_name = "prajjwal1/bert-tiny"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Sample input text
input_text = "The weather today is"
inputs = tokenizer(input_text, return_tensors="pt")

# Forward pass to get the logits
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Shape: [batch_size, sequence_length, vocab_size]

# Get probabilities for each token in the vocabulary at each position
probabilities = softmax(logits, dim=-1)  # Shape: [batch_size, sequence_length, vocab_size]

# Example: Probability distribution for the last token
last_token_prob = probabilities[0, -1, :]  # Probability distribution over vocabulary for the last token
top_5_tokens = torch.topk(last_token_prob, 5)  # Get top 5 tokens with highest probabilities

# Convert token IDs to words
top_5_tokens_words = [tokenizer.decode([token_id]) for token_id in top_5_tokens.indices]
top_5_probabilities = top_5_tokens.values.tolist()

# Output the results
print("Top 5 predicted tokens and their probabilities:")
for word, prob in zip(top_5_tokens_words, top_5_probabilities):
    print(f"Token: {word} - Probability: {prob:.4f}")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Top 5 predicted tokens and their probabilities:
Token: today - Probability: 0.1905
Token: . - Probability: 0.1574
Token: : - Probability: 0.1569
Token: the - Probability: 0.0707
Token: , - Probability: 0.0409
